In [29]:
import json

# log = "logs/gsm/gpt-4-turbo_1.0_naive_standard_sample_10_start0_end100.json"
log = "logs/gsm/gpt-4-turbo_1.0_naive_cot_sample_40_start22_end100.json"
# log = "logs/gsm/gpt-4-turbo_1.0_naive_cot_sample_40_start0_end100.json"
# log = "logs/gsm/gpt-3.5-turbo_1.0_naive_cot_sample_40_start0_end100.json"

with open(log) as f:
    data = json.load(f)

print(data)

[{'idx': 22, 'ys': ["Strategy:\nFirst, calculate the total amount of money Kelly has initially in cents. She has 5 quarters and each quarter is worth 25 cents; thus, 5 quarters equal 5 x 25 cents. Additionally, she has 2 dimes and each dime is worth 10 cents; thus, 2 dimes equal 2 x 10 cents. Add these amounts to find the total number of cents Kelly has initially.\nNext, subtract the cost of the can of pop, which is 55 cents, from the total amount of money to find out how much money she will have left.\n\nAnswer:\nKelly's total amount in cents = (5 x 25) + (2 x 10) = 125 + 20 = 145 cents.\nCost of the can of pop = 55 cents.\nMoney left after buying the pop = 145 cents - 55 cents = 90 cents.\nThus, the answer is 90.", "Strategy:\nTo find out how many cents Kelly will have left after buying the can of pop, I first need to calculate the total amount of money she has in cents. This involves determining the value of the quarters and the dimes in cents, then adding them together. After that,

In [35]:
rewards = [
    [data[i]['infos'][j]['r'] for j in range(len(data[0]['infos']))]
    for i in range(len(data))
]
reward_any = [any(rewards[j]) for j in range(len(rewards))]
sum(reward_any) / len(reward_any)

0.96

In [26]:
from datasets import load_dataset

dataset = load_dataset("openai/gsm8k", "main")
og_len = len(dataset["test"])
select_idxs = list(range(0, og_len, og_len // 100))
problems = [dataset["test"][i]["question"] for i in select_idxs]
answers = [
    dataset["test"][i]["answer"].split("\n")[-1][len("#### ") :]
    for i in select_idxs
]
dataset_answers = answers

In [27]:
import re
def find_answer(output: str):
    try:
        answer_idx = output.lower().index("the answer is ")
    except ValueError:
        return None
    answer = output[answer_idx + len("the answer is ") :]
    try:
        answer = re.findall(r"\d+", answer)[0]
    except IndexError:
        return None
    return answer

In [3]:
# import json

# log = "logs/gsm/gpt-3.5-turbo_1.0_naive_cot_sample_40_start0_end100.json"
# with open(log) as f:
#     data = json.load(f)

# print(data)

[{'idx': 0, 'ys': ["Strategy:\n1. Calculate the total number of eggs Janet has left after breakfast and baking muffins.\n2. Subtract the number of eggs she eats and bakes from the total number of eggs laid to find out how many she sells.\n3. Multiply the number of eggs she sells by the price per egg to find out how much she makes daily at the farmers' market.\n\nAnswer:\nJanet has (16 - 3 - 4) = 9 eggs left to sell every day.\nShe makes $2 per egg at the farmers' market.\nTherefore, she makes 9 x $2 = $18 every day at the farmers' market. \nThe answer is $18.", "Strategy:\nCalculate the total number of eggs Janet sells at the farmers' market per day, then multiply that by the price per egg.\n\nAnswer:\nJanet sells (16 - 3 - 4) = 9 eggs per day at the farmers' market.\nShe makes $2 per egg, so she makes 9 * $2 = $18 at the farmers' market every day.\nTherefore, the answer is $18.", "Strategy:\n1. Calculate the total number of eggs Janet has available to sell each day by subtracting the 

In [31]:
_dataset_answers = dataset_answers
dataset_answers = _dataset_answers[22:100]


In [34]:
from collections import Counter

correct = 0
N = 5
for i in range(len(data)):
    answers = [find_answer(y) for y in data[i]['ys'][:N]]
    answers = [a for a in answers if a is not None]
    # get the most common answer
    most_common = Counter(answers).most_common(1)[0][0]

    correct += (most_common == dataset_answers[i])

print(correct / len(data), correct, len(data))
correct / len(data)


0.9375 30 32


0.9375